# `euclidean_trick` with numexpr

Modify the `euclidean_trick_numexpr` function on the 3rd cell to use numexpr. Time it and compare that the result is the same as `euclidean_trick`.

In [ ]:
import numpy as np
import numexpr as ne

In [ ]:
def euclidean_trick(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

The most relevant part that can be replaced with `numexpr` is `np.abs(x2 + y2 - 2. * xy)`.

In [ ]:
def euclidean_trick_numexpr(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]

    xy = np.dot(x, y.T)

    return ne.evaluate('abs(x2 + y2 - 2. * xy)')

>> Although not really necessary, it may seem a good idea to replace the `einsum`s by numexpr's reductions. However, that may harm the performance: According to [issue#73](https://github.com/pydata/numexpr/issues/73) in numexpr's repository, it seems there is a long-time known problem with numexper `sum` reduction. For our case it wouldn't change too much the results since the array is not to large in the axis whe we are reducing.

In [ ]:
nsamples = 6000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

In [ ]:
%timeit euclidean_trick(x, x)
%timeit euclidean_trick_numexpr(x, x)

In [ ]:
print(np.abs(euclidean_trick(x, x) - euclidean_trick_numexpr(x, x)).max())
print(np.abs(euclidean_trick(x, x) - euclidean_trick_numexpr_redu(x, x)).max())